## The journey from sucky precision and recall to not sucky 

Model 1:

40: ( HaveSimilarTitles(P1,P2) & BlocksPubs(P1,B) & BlocksPubs(P2,B) & (P1-P2) ) >> SamePub(P1,P2) ^ 2  
["SimilarTitles"]

40: ( HaveSimilarNames(A1,A2) & BlocksAuthors(A1,B) & BlocksAuthors(A2,B) & (A1-A2) ) >> SameAuthor(A1,A2) ^ 2  
["SimilarNames"]

40: ( ~HaveSimilarAuthors(P1,P2) & BlocksPubs(P1,B) & BlocksPubs(P2,B) & (P1-P2) ) >> ~SamePub(P1,P2) ^ 2  
["NotSimilarAuthors"]

20: ( SamePub(P1,P2) & HasAuthor(P1,A1) & HasAuthor(P2,A2) & HaveSimilarNames(A1,A2) & BlocksPubs(P1,PB) & BlocksPubs(P2,PB) & BlocksAuthors(A1,AB) & BlocksAuthors(A2,AB) & (P1-P2) & (A1-A2)) >> SameAuthor(A1,A2) ^ 2  
["SamePubSameAuthor"]

20: ( AreCoAuthors(A1,A2) & AreCoAuthors(A3,A4) & SameAuthor(A1,A3) & HaveSimilarNames(A2,A4) & BlocksAuthors(A1,B) & BlocksAuthors(A2,B) & BlocksAuthors(A3,B) & BlocksAuthors(A4,B) & (A1-A2) & (A3-A4) & (A1-A3) & (A2-A4)) >> SameAuthor(A2,A4) ^ 2  
["Co-occurrence"]

1: ( (P1-P2) & BlocksPubs(P1,B) & BlocksPubs(P2,B)) >> ~SamePub(P1,P2) ^ 2  
["Prior"]

1: ( (A1-A2) & BlocksAuthors(A1,B) & BlocksAuthors(A2,B)) >> ~SameAuthor(A1,A2) ^ 2  
["Prior"]

```
Stats for Author
Positive Class: P 0.0806, R 0.8138
Negative Class: P 0.2527, R 0.0067

Stats for Publications
Positive Class: P 0.0022, R 0.1099
Negative Class: P 0.8532, R 0.0938
```

### Did you add explicit zero truths to target variables? - Eriq

#### :/ Now I did.

```
Stats for Author
Positive Class: P 0.8577, R 0.8139
Negative Class: P 0.9797, R 0.9852

Positive Class: P 0.1781, R 0.8579
Negative Class: P 0.9972, recall 0.9283
```

### Didn't transitivity help?

Model 2:

Model 1 with

40: ( SamePub(P1,P2) & SamePub(P2,P3) & (P1-P3) & BlocksPubs(P1,B) & BlocksPubs(P2,B) & BlocksPubs(P3,B)) >> SamePub(P1,P3) ^ 2  
["Transitivity Publications"]

40: ( SameAuthor(A1,A2) & SameAuthor(A2,A3) & (A1-A3) & BlocksAuthors(A1,B) & BlocksAuthors(A2,B) & BlocksAuthors(A3,B)) >> SameAuthor(A1,A3) ^ 2  
["Transitivity Authors"]

```
Stats for Author
Positive Class: P 0.8574, R 0.9442
Negative Class: P 0.9938, R 0.9828

Stats for Publications
Positive Class: P 0.1871, R 0.8579
Negative Class: P 0.9973, R 0.9325
```

#### Not by much. Most significantly, recall for authors improved.

### What about the threshold? What does the inferred values look like?

Well, first off, it's low precision, so it can't be that the threshold is *low*. And indeed it isn't. Of 9900 publication pairs, there are 176 positive pairs, and our inference has 906 pairs with non-zero truth values, most of them over 0.5. More than the threshold being low, looks like we should be increasing the negative prior weight?

The current weight seems to work decently for authors though...

### It's the similarity metric isn't it?

One look at the rules and you realize that "wait a minute, there's only one rule that gives any evidence towards a positive same publication", the similar titles rule. And one glance at the file shows that it gives out scores very generously...

Well for one thing, the library (pyjarowinkler) has incorrectly named functions. If it's a distance measure, dist(a, a) should equal 0, but in this library's case equals 1.

Keeping in mind that it is named distance but really is similarity, here are some examples:

```
improving performance in neural networks using a boosting algorithm. 
     vs 
improving performance in neural networks using a boosting algorithm.
sim: 1.0

improving performance in neural networks using a boosting algorithm. 
     vs 
inference of finite automata using homing sequences.
sim: 0.75
```

This is weird if you consider the following:

```
>>> print distance.get_jaro_distance("hello", "haloa", winkler=True, scaling=0.1)
0.76
```

Adding noise improves the similarity :/

```
improving performance in neural networks using a boosting algorithm. 
     vs 
inference of finite automata using homing sequences. in r.l.
sim: 0.78
```



### But that's OK.

All literature makes mention of the fact that there is a step where you have to decide a threshold for any similarity metric, above which things can be considered coreferent. Perhaps that threshold is really high if the JW distance is the metric used.

At this point, we have a couple options:
1. Discretize the output of the get_jaro_distance function, esp. for publication titles by choosing a threshold as others do
2. Use a soft TF-IDF distance measure for titles as others do

Question: Can PSL do anything about this? What comes to mind is lowering the weight of the rule, but that's being unfair to the rule, in my opinion, if you want to maintain the semantics. Increasing the negative prior? I don't know, makes it feel like a hack to make allowance for something that isn't really doing *its* job well (I mean the distance metric).

### Using TFIDF 

```
improving performance in neural networks using a boosting algorithm. 
     vs 
inference of finite automata using homing sequences.
sim: 0.0818

improving performance in neural networks using a boosting algorithm. 
     vs 
improving performance in neural networks using a boosting algorithm.
sim: 1.0

improving performance in neural networks using a boosting algorithm. 
     vs 
inference of finite automata using homing sequences. in r.l.
sim: 0.1472
```

Much better, what? Why adding in r.l. should improve similarity, I don't understand. But oh well. It's acceptably low still.

### Did the results improve?

#### Of course they did.

```
Without Trans:

Stats for Author
Positive Class: P 0.8577, R 0.8139
Negative Class: P 0.9797, R 0.9852

Stats for Publications
Positive Class: P 0.9589, R 0.7955
Negative Class: P 0.9963, R 0.9994
```

Notice that the recall went down a few points though

```
With Trans:

Stats for Author
Positive Class: P 0.8571, R 0.9419
Negative Class: P 0.9936, R 0.9828

Stats for Publications
Positive Class: P 0.9589, R 0.7955
Negative Class: P 0.9963, R 0.9994
```

Transitivity seems to be doing nothing for publications, it would seem based on these results...